<h1 style="color:#fcba03;">💰📈AI Agent RAG</h1>

In [1]:
!pip install dateparser -q

In [2]:
import warnings
warnings.filterwarnings("ignore")
from RAGWorkflow import *
from langchain_community.llms import Ollama
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from ekonomim_com_gold_news_web_scraping import *
from bigpara_hurriyet_com_tr_gold_news_web_scraping import *
from Measure import *
from sklearn.metrics.pairwise import cosine_similarity
import ollama

In [3]:
print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	check_updates(check_updates)
	fetch_doviz(fetch_doviz)
	fetch_cnn(fetch_cnn)
	fetch_ekonomim(fetch_ekonomim)
	fetch_bigpara(fetch_bigpara)
	run_rag(run_rag)
	__end__([<p>__end__</p>]):::last
	__start__ --> check_updates;
	check_updates --> fetch_cnn;
	fetch_bigpara --> run_rag;
	fetch_cnn --> fetch_doviz;
	fetch_doviz --> fetch_ekonomim;
	fetch_ekonomim --> fetch_bigpara;
	run_rag --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



<h1 style="color: #fcd703;"> 🔍🧾🧾📐✍️✅RAG Sonucu </h1>

In [4]:
llm = Ollama(model="gemma:2b", temperature=0.3)
embeddings = OllamaEmbeddings(model="embeddinggemma:latest")

rag = AgenticRAG(llm, embeddings)
# "Son 3 güne ait gram altın haberlerini özetle."
# "Geçen haftaya ait gram altın haberlerini özetle."
rag.create_vector_db()
# "Bu haftaya ait gram altın haberlerini özetle."
input_state = {
    "question": "Bu haftaya ait gram altın haberlerini özetle.",
    "system_prompt": None,
    "doviz_update_needed": False,
    "cnn_update_needed": False,
    "ekonomim_update_needed": False,
    "bigpara_update_needed": False,
    "doviz_day_diff": 0,
    "cnn_day_diff": 0,
    "ekonomim_day_diff": 0,
    "bigpara_day_diff": 0,
    "rag_instance": rag
}

result = graph.invoke(input_state)

C:\Users\monoz\AppData\Local\Temp\ipykernel_47376\1928927988.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma:2b", temperature=0.3)


Toplam kayıt: 0
CNN güncel haberler indiriliyor ...
[BİLGİ] 15 haber adayı bulundu. İşleme başlanıyor...
cnn_altin_haber_listesi_json : li_tags length : 15
Cnn altin Title : 2 OCAK 2026 CANLI ALTIN FİYATLARI: Gram Altın Ne Kadar? Çeyrek Altın, Yarım Altın, Cumhuriyet Altını Ne Kadar? 2 Ocak Cuma Canlı Altın Kuru... link : https://finans.cnnturk.com/finans-haberler/altin/galeri/2-ocak-2026-canli-altin-fiyatlari-gram-altin-ne-kadar-ceyrek-altin-yarim-altin-cumhuriyet-altini-ne-kadar-2-ocak-2380329
Cnn altin Title : 1 OCAK 2026 CANLI ALTIN FİYATLARI: Gram Altın Ne Kadar? Çeyrek Altın, Yarım Altın, Cumhuriyet Altını Ne Kadar? 1 Ocak Perşembe Canlı Altın Kuru... link : https://finans.cnnturk.com/finans-haberler/altin/galeri/1-ocak-2026-canli-altin-fiyatlari-gram-altin-ne-kadar-ceyrek-altin-yarim-altin-cumhuriyet-altini-ne-kadar-1-ocak-2379834
Cnn altin Title : 31 ARALIK 2025 CANLI ALTIN FİYATLARI: Gram Altın Ne Kadar? Çeyrek Altın, Yarım Altın, Cumhuriyet Altını Ne Kadar? 31 Aralık Çarşamba

<h1 style="color: #00BFFF;"> 📊🎯Roberta Dil Modeli İle Benzerlik Ölçümü </h1>

In [5]:
def roberta_similarity(filtered_documents, result):
    similarity_model = RobertaSimilarity()
    fetched_documents = list(map(preprocess_text, filtered_documents))
    generated_response = preprocess_text(result)
    avg_similarity = similarity_model.average_similarity(fetched_documents, generated_response)
    print(f"Ortalama Cosine Similarity: {avg_similarity:.4f}")
    return avg_similarity

In [29]:
"""sum_similarity = 0
avg_similarity = roberta_similarity(rag.filtered_documents, result["response"])
sum_similarity += avg_similarity"""
 # From last week result  = Ortalama Cosine Similarity: 0.9943

Ortalama Cosine Similarity: 0.9943


In [6]:
sum_similarity = 0
avg_similarity = roberta_similarity(rag.filtered_documents, result["response"])
sum_similarity += avg_similarity
# Current week result is = Ortalama Cosine Similarity: 0.9983

Ortalama Cosine Similarity: 0.9983


<h1 style="color: #5F9EA0;"> 📊🎯Qwen Dil Modeli İle Benzerlik Ölçümü </h1>

In [7]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import torch.nn.functional as F

class QwenSimilarity:
    def __init__(self, model_name: str = "intfloat/multilingual-e5-small"):
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=True
        )
        self.model = AutoModel.from_pretrained(
            model_name,
            trust_remote_code=True
        )
        self.model.eval()

    def _embed(self, text: str) -> np.ndarray:
        inputs = self.tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512
        )

        with torch.no_grad():
            outputs = self.model(**inputs)

        # ✅ Qwen için DOĞRU: Mean Pooling
        embeddings = outputs.last_hidden_state.mean(dim=1)

        # ✅ Cosine similarity için L2 normalize
        embeddings = F.normalize(embeddings, p=2, dim=1)

        return embeddings.cpu().numpy()

    def average_similarity(self, documents: list[str], answer: str) -> float:
        """
        documents: List[str] -> RAG'den gelen doküman metinleri
        answer: str -> LLM tarafından üretilen cevap
        """
        answer_embedding = self._embed(answer)

        similarities = []
        for doc in documents:
            doc_embedding = self._embed(doc)
            sim = cosine_similarity(doc_embedding, answer_embedding)[0][0]
            similarities.append(sim)

        return float(np.mean(similarities))

In [8]:
def qwen_similarity(filtered_documents, result):
    similarity_model = QwenSimilarity()
    fetched_documents = list(map(preprocess_text, filtered_documents))
    generated_response = preprocess_text(result)
    avg_similarity = similarity_model.average_similarity(fetched_documents, generated_response)
    print(f"Ortalama Cosine Similarity: {avg_similarity:.4f}")
    return avg_similarity
avg_similarity = qwen_similarity(rag.filtered_documents, result["response"])
sum_similarity += avg_similarity
# Current week result is = Ortalama Cosine Similarity: 0.8802

Ortalama Cosine Similarity: 0.8802


<h1 style="color: #40E0D0;"> 📊🎯Ollama Dil Modeli İle Benzerlik Ölçümü </h1>

In [62]:
def chunk_text(text, chunk_size=1000):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

def ollama_similarity(filtered_documents, result):
    """
    preprocess_text: Callable[[str], str]
    filtered_documents: List[str]
    result: str (üretilen cevap)
    """
    # Dokümanları preprocess et
    documents = list(map(preprocess_text, filtered_documents))
    # Sonucu preprocess et
    generated_response = preprocess_text(result)
    # Response embedding
    response_embedding = ollama.embeddings(
        model="embeddinggemma:latest",
        prompt=generated_response
    )
    response_vec = np.array(response_embedding["embedding"]).reshape(1, -1)
    similarities = []
    # Her doküman için cosine similarity
    for doc in documents:
        if (len(doc) != 0 and doc != ""):
            chunks = chunk_text(doc)
            sims = []
            for chunk in chunks:
                emb = ollama.embeddings(model="embeddinggemma:latest", prompt=chunk)
                vec = np.array(emb["embedding"]).reshape(1, -1)
                cosine_similarity_chunk = cosine_similarity(vec, response_vec)[0][0]
                if(cosine_similarity_chunk != np.nan):
                    sims.append(cosine_similarity_chunk)
            similarities.append(np.mean(sims))
    avg_similarity = float(np.mean(similarities))
    print(f"Ortalama Cosine Similarity: {avg_similarity:.4f}")
    return avg_similarity

In [66]:
"""avg_similarity = ollama_similarity(rag.filtered_documents, result["response"])
sum_similarity += avg_similarity
"""
# Last week result was = Ortalama Cosine Similarity: 0.7044

Ortalama Cosine Similarity: 0.7044


In [18]:
result["response"]

"Bu hafta, gram altın fiyatlarının aşağı yönlü hareketli olduğunu göstermektedir. Gram altın, kapalı çarşı altının sıkça alıp satıldığı yerlerde hareketliliğe sebep oluyor. Bu durum, altın fiyatlarını yükseltemeye nedendir. Ayrıca, peki, gram altın fiyatlarının 29 aralık 2025'teki değeri 6.224,55 TL'dır. Bu değeri, 29 aralık 2025'teki gram altın fiyatlarının en yüksek seviyesidir."

In [22]:
avg_similarity = ollama_similarity(rag.filtered_documents, result["response"])
sum_similarity += avg_similarity

Ortalama Cosine Similarity: 0.6109


<h2 style="color: #FF4500;"> 🎯Sonuçları Yorumlama: </h2>
 
- <p style="color: #FFA500; font-weight: bold;"> RoBERTa gibi embedding-odaklı modeller, semantik benzerlik ölçümü için özel olarak optimize edilmiştir. </p>
- <p style="color: #FFA500; font-weight: bold;">RAG’da üretim için kullandığım LLM embedding’leri ise birincil olarak üretim (generation) odaklıdır, benzerlik ölçümü ikincil bir yetenektir. </p>
- <p style="color: #FFA500; font-weight: bold;">Bundan dolayı aynı cevaplar ve vektör veritabanından çekilen belgeler arasındaki benzerlik için sonuçlar şu şekilde çıkmıştır: </p>
- <p style="color: #FFA500; font-weight: bold;">RoBERTa için = Ortalama Cosine Similarity: 0.99 ve üzeri </p>
- <p style="color: #FFA500; font-weight: bold;">Qwen için = Ortalama Cosine Similarity: 0.88 ve üzeri </p>
- <p style="color: #FFA500; font-weight: bold;">RAG'da cevap üretimi için kullanılan embedding modeli "embeddinggemma:latest" için ise 0.60-0.70 civarı benzerlik skorları elde edilmiştir. </p>

<h2 style="color: blue;"> Example 2 </h2>

In [23]:
llm = Ollama(model="gemma:2b", temperature=0.3)
embeddings = OllamaEmbeddings(model="embeddinggemma:latest")

rag = AgenticRAG(llm, embeddings)
# "Son 3 güne ait gram altın haberlerini özetle."
# "Geçen haftaya ait gram altın haberlerini özetle."
rag.create_vector_db()
# "Bu haftaya ait gram altın haberlerini özetle."
input_state = {
    "question": "Düne ait gram altın haberlerini özetle.",
    "system_prompt": None,
    "doviz_update_needed": False,
    "cnn_update_needed": False,
    "ekonomim_update_needed": False,
    "bigpara_update_needed": False,
    "doviz_day_diff": 0,
    "cnn_day_diff": 0,
    "ekonomim_day_diff": 0,
    "bigpara_day_diff": 0,
    "rag_instance": rag
}

graph.invoke(input_state)

Toplam kayıt: 0
CNN güncel haberler indiriliyor ...
[BİLGİ] 15 haber adayı bulundu. İşleme başlanıyor...
cnn_altin_haber_listesi_json : li_tags length : 15
Cnn altin Title : 2 OCAK 2026 CANLI ALTIN FİYATLARI: Gram Altın Ne Kadar? Çeyrek Altın, Yarım Altın, Cumhuriyet Altını Ne Kadar? 2 Ocak Cuma Canlı Altın Kuru... link : https://finans.cnnturk.com/finans-haberler/altin/galeri/2-ocak-2026-canli-altin-fiyatlari-gram-altin-ne-kadar-ceyrek-altin-yarim-altin-cumhuriyet-altini-ne-kadar-2-ocak-2380329
reccord exist çıkıyor : date_str: 2026-01-02-----title: Altın yeni yılın ilk işlem gününde yükselişte
❗ Atlanıyor (zaten mevcut): bigpara | 2026-01-02 - Altın yeni yılın ilk işlem gününde yükselişte...
reccord exist çıkıyor : date_str: 2026-01-02-----title: ALTIN FİYATLARI BUGÜN 2 OCAK 2026 CUMA (CANLI-ANLIK TAKİP) || Altın fiyatları ne kadar oldu? - Yeni yılın ilk işlem günü! Bugün 1 gram altın, çeyrek altın, Cumhuriyet altını ve ONS altın alış - satış fiyatı ne kadar? İşte Kapalı Çarşı gram a

{'question': 'Düne ait gram altın haberlerini özetle.',
 'response': "Düne ait gram altın haberlerini özetle:\n\n- Altın fiyatlarında 2025 yılı altının rekorlar kırarak yükseliş gösterdiği bir yıl olarak kayıtlara geçti.\n\n- Altın fiyatlarını destekleyen faktörler arasında ABD başkanı Donald Trump'ın agresif şekilde faiz indirimi ve jeopolitik risklerin devam etmesi yer alıyor.\n\n- Altın fiyatlarını etkileyen diğer faktörler arasında ise fed faiz oranlarını düşürmesi beklenirken, altın fiyatlarının yükselmesinin uygulandığı belirtiliyor.\n\n- Altın fiyatlarını etkileyen en önemli faktörler arasında şunlar yer alıyor:\n\n  - En yüksek yıllık performansını sergileyen yıllardan biri 1979 yılıdır.\n\n  - ABD'nin faiz indirimlerinin etkileyici olduğu belirtiliyor.\n\n  - Rusya ile Ukrayna olan güvenlik sorunları, altın fiyatlarını etkileyebilen bir faktör.\n\n  - Altın emtia piyasasının güvenli liman özelliğiyle öne çıkması, altın fiyatlarını etkileyebilen bir faktör.",
 'system_prompt': 

<h1 style="color: #00BFFF;"> 📊🎯Roberta Dil Modeli İle Benzerlik Ölçümü </h1>

In [24]:
sum_similarity = 0
avg_similarity = roberta_similarity(rag.filtered_documents, result["response"])
sum_similarity += avg_similarity
# Current week result is = Ortalama Cosine Similarity: 0.9983

Ortalama Cosine Similarity: 0.9981


<h1 style="color: #5F9EA0;"> 📊🎯Qwen Dil Modeli İle Benzerlik Ölçümü </h1>

In [25]:
avg_similarity = qwen_similarity(rag.filtered_documents, result["response"])
sum_similarity += avg_similarity

Ortalama Cosine Similarity: 0.8666


<h1 style="color: #40E0D0;"> 📊🎯Ollama Dil Modeli İle Benzerlik Ölçümü </h1>

In [26]:
avg_similarity = ollama_similarity(rag.filtered_documents, result["response"])
sum_similarity += avg_similarity

Ortalama Cosine Similarity: 0.6249


<h2 style="color: blue;"> Example 3 </h2>

In [27]:
llm = Ollama(model="gemma:2b", temperature=0.3)
embeddings = OllamaEmbeddings(model="embeddinggemma:latest")

rag = AgenticRAG(llm, embeddings)
# "Son 3 güne ait gram altın haberlerini özetle."
# "Geçen haftaya ait gram altın haberlerini özetle."
rag.create_vector_db()
# "Bu haftaya ait gram altın haberlerini özetle."
input_state = {
    "question": "son 3 güne ait gram altın haberlerini özetle.",
    "system_prompt": None,
    "doviz_update_needed": False,
    "cnn_update_needed": False,
    "ekonomim_update_needed": False,
    "bigpara_update_needed": False,
    "doviz_day_diff": 0,
    "cnn_day_diff": 0,
    "ekonomim_day_diff": 0,
    "bigpara_day_diff": 0,
    "rag_instance": rag
}

graph.invoke(input_state)

Toplam kayıt: 0
CNN güncel haberler indiriliyor ...
[BİLGİ] 15 haber adayı bulundu. İşleme başlanıyor...
cnn_altin_haber_listesi_json : li_tags length : 15
Cnn altin Title : 2 OCAK 2026 CANLI ALTIN FİYATLARI: Gram Altın Ne Kadar? Çeyrek Altın, Yarım Altın, Cumhuriyet Altını Ne Kadar? 2 Ocak Cuma Canlı Altın Kuru... link : https://finans.cnnturk.com/finans-haberler/altin/galeri/2-ocak-2026-canli-altin-fiyatlari-gram-altin-ne-kadar-ceyrek-altin-yarim-altin-cumhuriyet-altini-ne-kadar-2-ocak-2380329
reccord exist çıkıyor : date_str: 2026-01-02-----title: Altın yeni yılın ilk işlem gününde yükselişte
❗ Atlanıyor (zaten mevcut): bigpara | 2026-01-02 - Altın yeni yılın ilk işlem gününde yükselişte...
reccord exist çıkıyor : date_str: 2026-01-02-----title: ALTIN FİYATLARI BUGÜN 2 OCAK 2026 CUMA (CANLI-ANLIK TAKİP) || Altın fiyatları ne kadar oldu? - Yeni yılın ilk işlem günü! Bugün 1 gram altın, çeyrek altın, Cumhuriyet altını ve ONS altın alış - satış fiyatı ne kadar? İşte Kapalı Çarşı gram a

{'question': 'son 3 güne ait gram altın haberlerini özetle.',
 'response': "Son 3 gün boyunca gram altın haberlerini özetleyelim:\n\n* 31 aralık çarşamba, gram altın 5 bin 959 lira seviyesinde seyrediyor.\n* 30 aralık çarşamba, gram altın 6200 tl'nin altına geriledi.\n* 29 aralık çarşamba, gram altın 4398 dolar seviyesine yükseldi.\n* 28 aralık çarşamba, gram altın 4373 dolar seviyesine düştü.\n* 27 aralık çarşamba, gram altın 40.588,21 tl'ın satışa gönderildi.",
 'system_prompt': None,
 'doviz_update_needed': True,
 'cnn_update_needed': True,
 'ekonomim_update_needed': True,
 'bigpara_update_needed': True,
 'rag_instance': <RAGClass.AgenticRAG at 0x17bcbc7c040>}

<h1 style="color: #00BFFF;"> 📊🎯Roberta Dil Modeli İle Benzerlik Ölçümü </h1>

In [28]:
sum_similarity = 0
avg_similarity = roberta_similarity(rag.filtered_documents, result["response"])
sum_similarity += avg_similarity

Ortalama Cosine Similarity: 0.9982


<h1 style="color: #5F9EA0;"> 📊🎯Qwen Dil Modeli İle Benzerlik Ölçümü </h1>

In [29]:
avg_similarity = qwen_similarity(rag.filtered_documents, result["response"])
sum_similarity += avg_similarity

Ortalama Cosine Similarity: 0.8796


<h1 style="color: #40E0D0;"> 📊🎯Ollama Dil Modeli İle Benzerlik Ölçümü </h1>

In [30]:
avg_similarity = ollama_similarity(rag.filtered_documents, result["response"])
sum_similarity += avg_similarity

Ortalama Cosine Similarity: 0.6079


<h2 style="color: blue;"> Example 4 </h2>

In [33]:
llm = Ollama(model="gemma:2b", temperature=0.3)
embeddings = OllamaEmbeddings(model="embeddinggemma:latest")

rag = AgenticRAG(llm, embeddings)
# "Son 3 güne ait gram altın haberlerini özetle."
# "Geçen haftaya ait gram altın haberlerini özetle."
rag.create_vector_db()
# "Bu haftaya ait gram altın haberlerini özetle."
input_state = {
    "question": """son 5 güne ait gram altın haberlerini özetle. Gram altının ve çeyrek altının yaklaşık veya ortalama TL değerini """
    """ve ons altının yaklaşık veya ortalama kaç dolar olduğunu söyle.""",
    "system_prompt": None,
    "doviz_update_needed": False,
    "cnn_update_needed": False,
    "ekonomim_update_needed": False,
    "bigpara_update_needed": False,
    "doviz_day_diff": 0,
    "cnn_day_diff": 0,
    "ekonomim_day_diff": 0,
    "bigpara_day_diff": 0,
    "rag_instance": rag
}

graph.invoke(input_state)

Toplam kayıt: 0
CNN güncel haberler indiriliyor ...
[BİLGİ] 15 haber adayı bulundu. İşleme başlanıyor...
cnn_altin_haber_listesi_json : li_tags length : 15
Cnn altin Title : 2 OCAK 2026 CANLI ALTIN FİYATLARI: Gram Altın Ne Kadar? Çeyrek Altın, Yarım Altın, Cumhuriyet Altını Ne Kadar? 2 Ocak Cuma Canlı Altın Kuru... link : https://finans.cnnturk.com/finans-haberler/altin/galeri/2-ocak-2026-canli-altin-fiyatlari-gram-altin-ne-kadar-ceyrek-altin-yarim-altin-cumhuriyet-altini-ne-kadar-2-ocak-2380329
reccord exist çıkıyor : date_str: 2026-01-02-----title: Altın yeni yılın ilk işlem gününde yükselişte
❗ Atlanıyor (zaten mevcut): bigpara | 2026-01-02 - Altın yeni yılın ilk işlem gününde yükselişte...
reccord exist çıkıyor : date_str: 2026-01-02-----title: ALTIN FİYATLARI BUGÜN 2 OCAK 2026 CUMA (CANLI-ANLIK TAKİP) || Altın fiyatları ne kadar oldu? - Yeni yılın ilk işlem günü! Bugün 1 gram altın, çeyrek altın, Cumhuriyet altını ve ONS altın alış - satış fiyatı ne kadar? İşte Kapalı Çarşı gram a

{'question': 'son 5 güne ait gram altın haberlerini özetle. Gram altının ve çeyrek altının yaklaşık veya ortalama TL değerini ve ons altının yaklaşık veya ortalama kaç dolar olduğunu söyle.',
 'response': 'Son 5 gün için gram altın ve çeyrek altının TL değerleri ve ons altının TL değerleri aşağıda verilmiştir:\n\n| Tarihi | Gram Altın | Çeyrek Altın | Ons Altın |\n|---|---|---|---|\n| 29.02 | 6.224,55 | 10.105,00 | 4.511,71 |\n| 28.02 | 6.249,70 | 10.114,00 | 4.530,88 |\n| 27.02 | 6.250,52 | 10.349,00 | 4.531,47 |\n| 26.02 | 6.225,35 | 10.114,00 | 4.512,53 |\n| 25.02 | 6.249,70 | 10.105,00 | 4.511,71 |',
 'system_prompt': None,
 'doviz_update_needed': True,
 'cnn_update_needed': True,
 'ekonomim_update_needed': True,
 'bigpara_update_needed': True,
 'rag_instance': <RAGClass.AgenticRAG at 0x17bdfe572e0>}

<h1 style="color: #00BFFF;"> 📊🎯Roberta Dil Modeli İle Benzerlik Ölçümü </h1>

In [34]:
sum_similarity = 0
avg_similarity = roberta_similarity(rag.filtered_documents, result["response"])
sum_similarity += avg_similarity

Ortalama Cosine Similarity: 0.9982


<h1 style="color: #5F9EA0;"> 📊🎯Qwen Dil Modeli İle Benzerlik Ölçümü </h1>

In [35]:
avg_similarity = qwen_similarity(rag.filtered_documents, result["response"])
sum_similarity += avg_similarity

Ortalama Cosine Similarity: 0.8833


<h1 style="color: #40E0D0;"> 📊🎯Ollama Dil Modeli İle Benzerlik Ölçümü </h1>

In [36]:
avg_similarity = ollama_similarity(rag.filtered_documents, result["response"])
sum_similarity += avg_similarity

Ortalama Cosine Similarity: 0.6250


<h2 style="color: blue;"> Example 5 </h2>

In [38]:
llm = Ollama(model="gemma:2b", temperature=0.3)
embeddings = OllamaEmbeddings(model="embeddinggemma:latest")

rag = AgenticRAG(llm, embeddings)
# "Son 3 güne ait gram altın haberlerini özetle."
# "Geçen haftaya ait gram altın haberlerini özetle."
rag.create_vector_db()
# "Bu haftaya ait gram altın haberlerini özetle."
input_state = {
    "question": "Son 2 haftaya ait gram altın haberlerini özetle.",
    "system_prompt": None,
    "doviz_update_needed": False,
    "cnn_update_needed": False,
    "ekonomim_update_needed": False,
    "bigpara_update_needed": False,
    "doviz_day_diff": 0,
    "cnn_day_diff": 0,
    "ekonomim_day_diff": 0,
    "bigpara_day_diff": 0,
    "rag_instance": rag
}

graph.invoke(input_state)

Toplam kayıt: 0
CNN güncel haberler indiriliyor ...
[BİLGİ] 15 haber adayı bulundu. İşleme başlanıyor...
cnn_altin_haber_listesi_json : li_tags length : 15
Cnn altin Title : 2 OCAK 2026 CANLI ALTIN FİYATLARI: Gram Altın Ne Kadar? Çeyrek Altın, Yarım Altın, Cumhuriyet Altını Ne Kadar? 2 Ocak Cuma Canlı Altın Kuru... link : https://finans.cnnturk.com/finans-haberler/altin/galeri/2-ocak-2026-canli-altin-fiyatlari-gram-altin-ne-kadar-ceyrek-altin-yarim-altin-cumhuriyet-altini-ne-kadar-2-ocak-2380329
reccord exist çıkıyor : date_str: 2026-01-02-----title: Altın yeni yılın ilk işlem gününde yükselişte
❗ Atlanıyor (zaten mevcut): bigpara | 2026-01-02 - Altın yeni yılın ilk işlem gününde yükselişte...
reccord exist çıkıyor : date_str: 2026-01-02-----title: ALTIN FİYATLARI BUGÜN 2 OCAK 2026 CUMA (CANLI-ANLIK TAKİP) || Altın fiyatları ne kadar oldu? - Yeni yılın ilk işlem günü! Bugün 1 gram altın, çeyrek altın, Cumhuriyet altını ve ONS altın alış - satış fiyatı ne kadar? İşte Kapalı Çarşı gram a

{'question': 'Son 2 haftaya ait gram altın haberlerini özetle.',
 'response': 'Son 2 hafta boyunca gram altın fiyatları, 27 aralık 2025 Cumartesi günü itibarıyla 4.530,88 dolar seviyelerine ulaştı. Çeyrek altın ise 10.236 lira ile yeni işlem gününe başladı. Gram altın, çeyrek altın, ons altın ve cumhuriyet altını ne kadar, kaç tl ediyor i şte 26 aralık 2025 serbest piyasa ve kapalıçarşı altın fiyatları 6.239,98 tl gram altın satış fiyatı, 6.240,76 tl çeyrek altin fi yatlari çeyrek altın alış fiyatı, 10.091,00 tl çeyrek altın satış fiyatı, 10.316,00 tl ons altin fi yatlari ons altın alış usd fiyatı, 4.520,75 ons altın satış usd fiyatı ve 4.521,28 cumhuri yet altini ne kadar, kaç tl cumhuriyet altını alış fiyatı.',
 'system_prompt': None,
 'doviz_update_needed': True,
 'cnn_update_needed': True,
 'ekonomim_update_needed': True,
 'bigpara_update_needed': True,
 'rag_instance': <RAGClass.AgenticRAG at 0x17bcbc7b760>}

<h1 style="color: #00BFFF;"> 📊🎯Roberta Dil Modeli İle Benzerlik Ölçümü </h1>

In [39]:
sum_similarity = 0
avg_similarity = roberta_similarity(rag.filtered_documents, result["response"])
sum_similarity += avg_similarity

Ortalama Cosine Similarity: 0.9967


<h1 style="color: #5F9EA0;"> 📊🎯Qwen Dil Modeli İle Benzerlik Ölçümü </h1>

In [40]:
avg_similarity = qwen_similarity(rag.filtered_documents, result["response"])
sum_similarity += avg_similarity

Ortalama Cosine Similarity: 0.8856


<h1 style="color: #40E0D0;"> 📊🎯Ollama Dil Modeli İle Benzerlik Ölçümü </h1>

In [63]:
avg_similarity = ollama_similarity(rag.filtered_documents, result["response"])
sum_similarity += avg_similarity

Ortalama Cosine Similarity: 0.6203


In [37]:
# end